<a href="https://colab.research.google.com/github/dcolinmorgan/DOTS/blob/main/featurize_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## adapted from https://jaketae.github.io/study/keyword-extraction/#candidate-selection
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from transformers import AutoModel, AutoTokenizer
import torch

model_name = "distilroberta-base"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
nlp = spacy.load('en_core_web_sm')
n_gram_range = (1, 2)
stop_words = "english"
embeddings=[]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

import nltk, string, numpy as np
nltk.download('punkt')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
device

device(type='cpu')

In [ ]:
def get_npr_stories(p):
    # Send a GET request to the NPR API
    r = requests.get("http://api.npr.org/query?apiKey=MDE5Mzg3Mjc2MDE0MzMyMjM3NjM5ZTI2Ng001", params=p)

    # Parse the XML response to get the story URLs
    root = ET.fromstring(r.content)
    story_urls = [story.find('link').text for story in root.iter('story')]

    # For each story URL, send a GET request to get the HTML content
    full_stories = []
    for url in story_urls:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the main content of the story. This will depend on the structure of the webpage.
        # Here, we're assuming that the main content is in a <p> tag. You might need to adjust this depending on the webpage structure.
        story = soup.find_all('p')

        # Extract the text from the story
        full_story = ' '.join(p.text for p in story)
        full_stories.append(full_story)
    return full_stories


In [ ]:
def chunk_text(text, max_len):
    # Tokenize the text into tokens
    tokens = nltk.word_tokenize(text)

    # Calculate the number of chunks and the size of the final chunk
    num_chunks = len(tokens) // max_len
    final_chunk_size = len(tokens) % max_len

    # If the final chunk is too small, distribute its tokens among the other chunks
    if final_chunk_size < max_len / 2:
        num_chunks += 1
        chunk_sizes = [len(tokens) // num_chunks + (1 if i < len(tokens) % num_chunks else 0) for i in range(num_chunks)]
        chunks = [tokens[sum(chunk_sizes[:i]):sum(chunk_sizes[:i+1])] for i in range(num_chunks)]
    else:
        chunks = [tokens[i:i + max_len] for i in range(0, len(tokens), max_len)]

    return chunks


In [ ]:
from torch.utils.data import DataLoader

def featurize_stories(text, max_len, top_k):
    # Extract candidate words/phrases
    count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([text])
    all_candidates = count.get_feature_names_out()
    doc = nlp(text)
    noun_phrases = set(chunk.text.strip().lower() for chunk in doc.noun_chunks)
    nouns = set()
    for token in doc:
        if token.pos_ == "NOUN":
            nouns.add(token.text)

    all_nouns = nouns.union(noun_phrases)
    candidates = list(filter(lambda candidate: candidate in all_nouns, all_candidates))
    candidate_tokens = tokenizer(candidates, padding=True, return_tensors="pt")
    candidate_tokens = {k: v.to(device) for k, v in (candidate_tokens).items()}
    candidate_embeddings = model(**candidate_tokens)["pooler_output"]
    candidate_embeddings = candidate_embeddings.detach()#.to_numpy()

    # words = nltk.word_tokenize(text)
    # chunks = [words[i:i + 512] for i in range(0, len(words), 512)]
    chunks = chunk_text(text, max_len)  # use this to chunk better and use less padding thus less memory but also less affect from averging

    for chunk in chunks:
        text_tokens = tokenizer(chunk, padding=True, return_tensors="pt")
        text_tokens = {k: v.to(device) for k, v in (text_tokens).items()}
        text_embedding = model(**text_tokens)["pooler_output"]
        text_embedding = text_embedding.detach()#.to_numpy()
        embeddings.append(text_embedding)
    max_emb_shape = max(embedding.shape[0] for embedding in embeddings)
    padded_embeddings = [np.pad(embedding.cpu(), ((0, max_emb_shape - embedding.shape[0]), (0, 0))) for embedding in embeddings]
    avg_embedding = np.min(padded_embeddings, axis=0)
    distances = cosine_similarity(avg_embedding, candidate_embeddings.cpu())
    torch.cuda.empty_cache()
    return [candidates[index] for index in distances.argsort()[0][::-1][-top_k:]]



In [ ]:
import pandas as pd
# data=pd.read_csv('/content/drive/MyDrive/consult/florida-hurricane-tweet.csv')
data=pd.read_csv('/content/drive/MyDrive/consult/Louie_california_weather.csv')

In [ ]:
data['text']

0       California Snow Storm Latest as Drivers Trappe...
1       Large, illegal marijuana grow posed ‘extreme f...
2       @nytimes The classic California extreme weathe...
3                extreme weather https://t.co/hp9yOVBkMR 
4                Extreme weather https://t.co/T5QtBjx3z4 
                              ...                        
9995    @cryptojack Conquer the web3 platform with $BL...
9996                  it’s a hockey game not desert storm
9997    @NHLFlames But we are having fun and we have a...
9998    @BlastRunnersPVP 3 days past after last announ...
9999                            @Balkaur07527817 Ok then?
Name: text, Length: 10000, dtype: object

In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/consult/florida-hurricane-tweet_features.txt',sep='\t')


In [ ]:
rank_articles=[]
from tqdm import tqdm
# for i in tqdm(range(len(df),len(data['text']))):
dataloader = DataLoader(data['text'], batch_size=32, shuffle=True, num_workers=4)
for i in tqdm(range(len(dataloader))):
    try:
        cc=featurize_stories(data['text'][i], max_len=512, top_k=4)
        # print(cc)
        rank_articles.append(cc)
    except IndexError:
        pass

  2%|▏         | 161/10000 [02:28<2:07:04,  1.29it/s]

In [ ]:
flattened_list = [item for sublist in rank_articles for item in sublist]
from collections import Counter
counter = Counter(flattened_list)
df = pd.DataFrame.from_dict(counter, orient='index', columns=['Count'])

df = df.sort_values(by='Count',ascending=False)
# df.to_csv('/content/drive/MyDrive/consult/florida-hurricane-tweet_features2.txt',sep='\t')
df.to_csv('/content/drive/MyDrive/consult/california_weather_tweet_features.txt',sep='\t')

print(len(df))
# df[:25]


In [ ]:
len(rank_articles)

In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/consult/florida-hurricane-tweet_features.txt',sep='\t')
# df2=pd.read_csv('/content/drive/MyDrive/consult/florida-hurricane-tweet_features2.txt',sep='\t')

# df=pd.concat([df,df2])
# print(df['Unnamed: 0'])
# df = df.groupby('Unnamed: 0').sum().sort_values(by='Count',ascending=False)
# df=df[df['Count']>int(np.round(len(df)*.001))]

In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')

# nouns = ['apple', 'John', 'London', 'dog', 'Mary', 'Paris', 'banana']
nouns= df.reset_index()['Unnamed: 0'].to_list()
doc = nlp(' '.join(nouns))

proper_nouns = [token.text for token in doc if token.pos_ == 'PROPN']

print(proper_nouns)

In [ ]:
print(len(proper_nouns))
proper_nouns

In [ ]:
len(nouns)

# run from module

In [3]:
# %%time
# !pip install --quiet git+https://github.com/dcolinmorgan/DOTS.git

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.6 MB/s eta 0:00:00
CPU times: user 563 ms, sys: 68.6 ms, total: 631 ms
Wall time: 1min 21s


In [26]:
import sys,os,argparse,csv
sys.argv = ['dots_feat.py', '-n', '100', '-f', '3', '-o', 'dots_feats.csv', '-s', '1']
parser = argparse.ArgumentParser(description='Process OS data for dynamic features.')
parser.add_argument('-n', type=int, default=10, help='Number of data items to get')
parser.add_argument('-f', type=int, default=3, help='Number of features per item to get')
parser.add_argument('-o', type=str, default='dots_feats.csv', help='Output file name')
parser.add_argument('-s', type=int, default=1, help='Parallelize requests')
args, unknown = parser.parse_known_args()

from DOTS.dots_feat import featurize_stories, process_data, get_data, process_hit

In [2]:
from google.colab import userdata
ost=userdata.get('OS_TOKEN')
text_file = open("/usr/local/lib/python3.10/dist-packages/DOTS/.env", "w")
text_file.write('OS_TOKEN='+ost)
text_file.close()

In [3]:
import logging
from tqdm import tqdm
logging.basicConfig(level=logging.INFO)

In [4]:
data = get_data(100)
articles = process_data(data)

grabbing text from url: 100%|██████████| 100/100 [01:22<00:00,  1.22it/s]


In [42]:
rank_articles=[]
for i in tqdm(articles):
    parts=str(i).split('[', 3)
    try:
        cc=featurize_stories(str(i), top_k = args.f, max_len=512)
        cc.append(parts[1])
        rank_articles.append(cc)
    except Exception as e:
         logging.error(f"Failed to process article: {e}")
with open(args.o, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(rank_articles)

 36%|███▌      | 35/98 [00:53<01:35,  1.52s/it]ERROR:root:Failed to process article: list index out of range
ERROR:root:Failed to process article: list index out of range
100%|██████████| 98/98 [02:28<00:00,  1.52s/it]


In [ ]:
rank_articles

In [45]:
flattened_list = [item for sublist in rank_articles for item in sublist]
list_without_brackets = [s.replace("[", "").replace("]", "") for s in flattened_list]

from collections import Counter
counter = Counter(list_without_brackets)

In [52]:
import pandas as pd
df = pd.DataFrame.from_dict(counter, orient='index', columns=['Count'])
df = df.sort_values(by='Count',ascending=False)
df = df[~df.index.str.contains("united states")]
df = df[~df.index.str.contains("United States")]
df = df[~df.index.str.contains("None")]

df[:25]

,Count
firefighters,13
afternoon,6
disease,5
temperatures,5
rebels,4
emergency services,4
morning,4
hearst television,3
ceasefire,3
heavy snow,3


In [66]:
filtered_list = [sublist for sublist in rank_articles if any('fire'.lower() in s.lower() for s in sublist)]


In [73]:
filtered_list

[['emergency services',
  'firefighters',
  'afternoon',
  "'Corofin, Clare, Ireland', '16-01-2024', "],
 ['defying order', 'councillors', 'ceasefire', "None, '16-01-2024', "],
 ['firefighters', 'soups', 'flashing staff', "None, '16-01-2024', "],
 ['unwanted wildfires',
  'foster parents',
  'disease',
  "'California, United States', '16-01-2024', "],
 ['highway oo', 'firefighters', 'temperatures', "None, '16-01-2024', "],
 ['deputies',
  'united states',
  'firefighters',
  "'Lewis County, New York, United States', '16-01-2024', "],
 ['super tuesday', 'voters', 'firefighters', "None, '16-01-2024', "],
 ['firefighters', 'okanagan lake', 'emergency crews', "None, '16-01-2024', "],
 ['ceremony',
  'ceasefire',
  'minneapolis',
  "'Philadelphia, Pennsylvania, United States', '16-01-2024', "],
 ['firefighters', 'lamp', 'nominee', "None, '16-01-2024', "],
 ['morning',
  'parents',
  'firefighters',
  "'Kaitangata, New Zealand (General), New Zealand', '16-01-2024', "],
 ['morning', 'firefigh

In [8]:
!git clone https://github.com/dcolinmorgan/DOTS.git
# !python DOTS/DOTS/dots_feat.py -n 100 -f 5 -s 1 -o dots_feat.txt

Cloning into 'DOTS'...
remote: Enumerating objects: 312, done.
remote: Counting objects: 100% (312/312), done.
remote: Compressing objects: 100% (196/196), done.
remote: Total 312 (delta 155), reused 228 (delta 78), pack-reused 0
Receiving objects: 100% (312/312), 8.53 MiB | 12.08 MiB/s, done.
Resolving deltas: 100% (155/155), done.


In [12]:
!pytest DOTS/DOTS/test_dots_feat.py

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0
rootdir: /content/DOTS
plugins: anyio-3.7.1
collected 3 items                                                                                  

DOTS/DOTS/test_dots_feat.py ...                                                        [100%]

======================================== 3 passed in 15.56s ========================================
